# Practicals I: Fitting DGMs and introducing structure into latent representations

In this first practical session, we will 
 * learn how to build a neural network in Julia
 * compare different dimension reduction methods on a single-cell RNA-seq dataset
 * build an autoencoder model that learns a low-dimensional representation of a single-cell RNA-seq dataset and compare its representation PCA and UMAP 
 * train the autoencoder to find a latent representation that resembles the ones found by UMAP or PCA
 * make it generative: turn the autoencoder into a VAE and sample synthetic data  
 * explore different loss functions and how different models are affected by hyperparameter choices

Bonus material: 
 * build a neural network from scratch 
 * train it to classify cells into GABA-ergic and Glutamatergic neurons based on their single-cell gene expression profile

Before we get started: 

## Acknowledgement

Huge thanks to our amazing Master's student **Niklas Brunn** for the great work on revising, extending and polishing this notebook!

## Setting up the environment

In the `Project.toml` and `Manifest.toml` file in the folder in the repository, all the necessary packages and their versions are specified. We don't have to manually install them, but can simply tell Julia to configure the environment as specified in these files, including installing all necessary dependencies. This can be done with the Julia package manager with the following commands: 

For more info on the Julia package manager, see [here](https://docs.julialang.org/en/v1/stdlib/Pkg/)

In [ ]:
using Pkg; # loading the package manager
cd(@__DIR__) # change into the directory where the current file (this notebook) is located
Pkg.activate(".") # activate the environment specified in the .toml files
Pkg.instantiate() # install the necessary dependencies
Pkg.status() # show which packages are currently installed in the environment

## Load the data 

We will use a dataset from Tasic et al. (2016), where gene expression is profiled in 1525 single neurons, extracted from the cortices of mice. The authors reported a set of 104 genes (Figure 3 of Tasic et al.), based on which different subtypes of neurons can be discriminated and a set of 80 neurotransmitter receptor genes (Figure S15 of Tasic et al.), totalling to 184 genes. 

The data from Tasic et al. (2016) has been stored in the [Gene expression Omnibus (GEO)](https://www.ncbi.nlm.nih.gov/geo/) under accession number GSE71585. 

We have downloaded the expression data and the correspoding sample description sheet from the GEO and removed the samples from the expression data which have been flagged by Tasic et al. (2016) as being of bad quality. Since the samples of bad quality are not contained within the sample description sheet, we can filter the samples of bad quality by comparing the sample IDs in the expression data with the IDs in the sample description sheet. 

Further, we have removed genes which are expressed with less than 10 aligned reads in less than 20 samples. This is necessary since many genes are usually expressed only in a single cell and consequently these genes do not substantially represent the structure in the data.

Next, the data has been normalized for sequencing depth using an algorithm from DESeq (Anders and Huber, 2010).
Normalization for sequencing depth is important since the number of aligned reads per sample fluctuate which affects the estimated expression level for each gene. The algorithm of Anders and Huber (2010) calculates correction factors, one for each sequencing library, i.e. sample, by which the sequencing data are divided. 

The data contains expression levels of over 15.119 genes in 1679 samples. To lower computational demands of the following analysis while still maintaining most of the biological signal int he dataset, we have subsetted the data using a set of genes identified as most relevant by Tasic et al. (2016): Specifically, we use the marker genes which are known to indicate cell type membership ([Figure 3](https://www.nature.com/articles/nn.4216/figures/3) in Tasic et al. (2016)) and the neurotransmitter receptorgenes which are differentially expressed in different neuron cell types but have a less pronounced expression pattern (supplementary material, [Figure S 15](https://static-content.springer.com/esm/art%3A10.1038%2Fnn.4216/MediaObjects/41593_2016_BFnn4216_MOESM67_ESM.pdf))
 
Finally, we selected only the neural cells using the cell type annotation provided by Tasic et al. (2016), and annotated the cell types as GABAergic vs. Glutamatergic. 

The complete preprocessing can be reproduced by running the script `preprocess_Tasic.jl`, where at the end the following four files are saved to the `data` subfolder: 

 * `genes.txt` - list of selected genes 
 * `single_cell_mat` - the normalized count matrix, subsetted to the relevant genes and neural cells 
 * `celltype.txt` - the cell type annotation of the cells in `single_cell_mat.txt`, as provided by Tasic et al. (2016)
 * `gabagluta.txt` - the annotation of cells as GABAergic vs. Glutamatergic. 

In [ ]:
using DelimitedFiles
genenames = readdlm("data/genes.txt")
mat_norm = readdlm("data/single_cell_mat.txt")
celltype = readdlm("data/celltype.txt")
gabagluta = readdlm("data/gabagluta.txt")

We can check how many genes and cells we have, and some more information about the number of different cell types and Gaba- vs Glutamatergic cells: 

In [ ]:
ngenes, ncells = size(mat_norm)
ncelltypes = length(unique(celltype))
ngaba = length(gabagluta[gabagluta.=="GABA"])
ngluta = length(gabagluta[gabagluta.=="Glutamate"])

println("Data matrix is of size ", size(mat_norm))
println("Number of genes is ", length(genenames))
println("Number of cells is ", length(celltype))

println("There are $ncelltypes different cell types")
println("There are $ngaba GABA-ergic cells and $ngluta Glutamatergic cells")

## Some exploratory visualization 

Now that we have loaded the data and checked some basic dimensions, let's take a look at it! 

We start by looking at the distribution of overall counts per gene.

- exemplary distribution of some genes across GABA vs Gluta cells
- some summary statistics 

In [ ]:
countspergene = mapslices(x -> sum(x), mat_norm, dims=2)
using VegaLite
@vlplot(:bar, 
    x={log.(vec(countspergene)), bin={step=1, extent=[0,18]}, title="log(sum of counts across all cells)"}, 
    y={"count()", title="number of genes"},
    width=500,
    height=200
)

Now, we can also split this into GABA-ergic and Glutamatergic cells:

In [ ]:
gabainds = findall(x -> x == "GABA", vec(gabagluta))
glutainds = findall(x -> x == "Glutamate", vec(gabagluta))
data_gaba = mat_norm[:,gabainds]
data_gluta = mat_norm[:,glutainds]

countspergene_gaba = mapslices(x -> sum(x), data_gaba, dims=2)
countspergene_gluta = mapslices(x -> sum(x), data_gluta, dims=2)

countspergene_gabagluta = vcat(countspergene_gaba, countspergene_gluta)
gabagluta_info = vec(vcat(fill("GABA", ngenes), fill("Glutamate", ngenes)))

@vlplot(:bar, 
    x={log.(vec(countspergene_gabagluta)), bin={step=1, extent=[0,18]}, title="log(sum of counts across all cells)"}, 
    y={"count()", title="number of genes"},
    column = gabagluta_info,
    color = gabagluta_info,
    width=230,
    height=200
)

Next, let's try to look at the structure in the data by applying some dimension reduction. We will calculate PCA and UMAP coordinates of the data and plot. 

First, we start off by applying PCA. Since PCA expects symmetric data, we need to standardize it, after log-transforming and adding a pseudo-count of 1 to deal with the over-dispersion and skewness. 

Let's first define the required functions:

In [ ]:
# standardize: subtract mean and divide by standard deviation 
function standardize(x)
    (x .- mean(x, dims = 1)) ./ std(x, dims = 1)
end

# function to compute principal components based on singular value decomposition
using LinearAlgebra
using StatsBase
function prcomps(mat)
    u,s,v = svd(mat)
    prcomps = u * Diagonal(s)
    return prcomps
end

Now that we have defined the functions, we can apply PCA to the data matrix. Note that we have defined the `prcomps` function to calculate the principal components such that it expects the features (=genes) to be the *columsn* of the data matrix, while currently the genes are the rows. Thus, we transpose the standardized and log-transformed data before passing it to the `prcomps` function. 

In [ ]:
input_mat = standardize(log.(mat_norm .+ 1)') # ' realises matrix transpose
pcs = prcomps(input_mat)

Let's look at the first 2 principal components and visualize them. 

In [ ]:
using DataFrames
dimred_df = DataFrame(PC1 = pcs[:,1], PC2 = pcs[:,2])
dimred_df |> 
@vlplot(:point, 
    x = "PC1", y="PC2",
    width=400, height=250,
    title="First 2 principal components"
)

Ok great! There seem to be some groups in the data. Let's add cell type and GABA-ergic/Glutamatergic info to the plot so we can see whether the clusters correspond to biologically meaningful groups of cells. 

In [ ]:
dimred_df[!,:celltype] = vec(celltype)
dimred_df |> 
@vlplot(:point, 
    x = :PC1, y=:PC2,
    color={:celltype, title="Cell type", scale={scheme="category20"}},
    title="PCs color-coded by cell type",
    width=400, height=250
)

In [ ]:
dimred_df[!,:gabagluta] = vec(gabagluta)
dimred_df |> 
@vlplot(:point, 
    x = :PC1, y=:PC2,
    color={:gabagluta, title="GABAergic vs Glutamatergic"},
    title="PCs color-coded by GABA-ergic vs Glutamatergic cells",
    width=400, height=250
)

Now, let's compare this to UMAP coordinates. We use the Julia implementation in the `UMAP.jl` package.
As for PCA, we use the log-transformed and standardized data matrix `input_mat` as input (Unlike PCA, UMAP expects the data in a (`feature` x `observations`) format, hence the double transpose): 

In [ ]:
using Random
using UMAP 
Random.seed!(42); # for reproducibility 
umap_coords = umap(input_mat', n_neighbors=70, min_dist=1.5)' 

We can now add the UMAP-coordinates to our `dimred_df`.

In [ ]:
dimred_df[!,:UMAP1] = umap_coords[:,1]
dimred_df[!,:UMAP2] = umap_coords[:,2]

dimred_df

For plotting, again using color-coding for the celltype and GABA- vs. Glutamatergic cells, we re-structure our data frame a bit. 

In [ ]:
dimred_stacked = DataFrame(dim1 = vcat(pcs[:,1], umap_coords[:,1]),
                        dim2 = vcat(pcs[:,2], umap_coords[:,2]),
                        celltype = repeat(vec(celltype), outer=2),
                        gabagluta = repeat(vec(gabagluta), outer=2),
                        method = repeat(["PCA", "UMAP"], inner=size(pcs,1))
)
dimred_stacked |> 
@vlplot(:point,
    x={:dim1, title=""}, y={:dim2, title=""},
    color={:celltype, title="cell type", scale={scheme="category20"}},
    column={:method, title=""},
    resolve={scale={x="independent",y="independent"}} #so axes can have different scales for each method   
)

> **Exercise:**
>
> Make the same plot as above but use the `:gabagluta` annotation to color the cells, to see how well the two types separate!

## How deep learning in Julia works

Now that we've gotten an idea of what's in our data, we will learn how to build a simple neural network from scratch in Julia. 

There is also a package `Flux.jl` where many building blocks, training routines and data handling functions are built-in. Later on in the notebook, we will use this package, but for this first exercise we will learn how to build a network from scratch. This is also pretty much what `Flux.jl` does under the hood.

We have seen so far that all dimension reduction methods do a good job at differentiating between the GABA- and the Glutamatergic cells. 
For this first exercise to serve as an instructive example of how to build and train a neural network, we will thus train a classifier to automatically assign the cells into GABA-ergic and Glutamatergic neurons. 

### Defining neural networks

Mathematically speaking, a neural network is essentially simply a function composition, consisting of both linear and non-linear functions. Each linear function is defined as multiplication by some parameter matrix called the *weights* of the network, and addition of an offset parameter vector called the *bias*. 

Denoting the weight matrix as $W \in \mathbb{R}^{n \times p}$ and the bias vector as $b \in \mathbb{R}^p$, the linear transformation of an input vector $x \in \mathbb{R}^p$ can be written as $W\cdot x + b$. Such a linear transformation is typically followed by a non-linear *activation function* that is applied element-wise. Commonly used activation functions are the relu, sigmoid or hyperbolic tangent function. 

Denoting the activation function as $\phi: \mathbb{R} \to \mathbb{R}$, one layer of a neural network can thus be defined as 
$ \phi.(W\cdot x + b) $, where $.$ denotes element-wise application of $\phi$. 

A *deep* neural network now simply consists of several such layers stacked on top of each other. A network with $K$ layers can thus be represented as a function $f_{\mathrm{NeuralNet}}(x) := f_K \circ f_{K-1} \circ ...\circ f_2 \circ f_1$, where each $f_k$ is of the form $f_k = \phi_k.(W_k \cdot x_k + b_k)$.

Data is processed throught the network by subsequently applying these layer functions. A neural network architecture and forward propagation is illustrated in the following graphic: 

![](figures/NNForwardPropDetails.png)

(Figure adapted from [Angermueller et al. (2016) Deep learning for computational biology, Molecular Systems Biology.](https://www.embopress.org/doi/full/10.15252/msb.20156651))

In `Flux.jl`, the `Dense` struct implements a fully-connected layer of exactly the form as described above: 

In [ ]:
using Flux
?Flux.Dense

````
help?> Dense
search: Dense DenseArray DenseVector DenseMatrix DenseVecOrMat DenseConvDims

  Dense(in => out, σ=identity; bias=true, init=glorot_uniform)
  Dense(W::AbstractMatrix, [bias, σ])

  Create a traditional fully connected layer, whose forward pass is given by:

  y = σ.(W * x .+ bias)

  The input x should be a vector of length in, or batch of vectors represented
  as an in × N matrix, or any array with size(x,1) == in. The out y will be a
  vector of length out, or a batch with size(y) == (out, size(x)[2:end]...)

  Keyword bias=false will switch off trainable bias for the layer. The
  initialisation of the weight matrix is W = init(out, in), calling the
  function given to keyword init, with default glorot_uniform. The weight
  matrix and/or the bias vector (of length out) may also be provided
  explicitly.

  Examples
  ≡≡≡≡≡≡≡≡≡≡

  julia> d = Dense(5 => 2)
  Dense(5 => 2)       # 12 parameters

  julia> d(rand(Float32, 5, 64)) |> size
  (2, 64)

  julia> d(rand(Float32, 5, 1, 1, 64)) |> size  # treated as three batch dimensions
  (2, 1, 1, 64)

  julia> d1 = Dense(ones(2, 5), false, tanh)  # using provided weight matrix
  Dense(5 => 2, tanh; bias=false)  # 10 parameters

  julia> d1(ones(5))
  2-element Vector{Float64}:
   0.9999092042625951
   0.9999092042625951

  julia> Flux.params(d1)  # no trainable bias
  Params([[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]])
````

It can be constructed by passing the input and output dimensions `n` and `p`, i.e., the dimensions of the weight matrix, and the activation function: 

In [ ]:
l = Dense(10, 20, relu)

Multiple layers can be combined into a `Chain`: 

In [ ]:
neural_network = Chain(Dense(10, 20, relu), Dense(20, 40, tanh), Dense(40, 2, σ))

### Training neural networks

All these parameters need to be optimised, in order for our neural network to learn something meaningful. 
Training a neural network means to find values for the weights and biases of all layers, such that the overall neural network approximates some function of interest.

For this, in addition to the neural network model itself, we need to define a loss function that measures how good our current approximation is. We can then minimize the loss function with respect to the network parameters, the weights and biases, using a version of gradient descent. 

Gradient descent is an numerical optimization algorithm where the network parameters get iteratively updated by adding the weighted negativ gradient of the network, with respect to the network parameters, to each parameter. The gradient is calculated via backpropagation (reverse mode of automatic differentiation). 

The following images show the optimization process of a neural network via gradient descent:

![](figures/ForwardAndBackwardProp.png)
![](figures/GradientDescent.png)

(Figure adapted from [Angermueller et al. (2016) Deep learning for computational biology, Molecular Systems Biology.](https://www.embopress.org/doi/full/10.15252/msb.20156651))

## Building an autoencoder

Now that we know how to build simple models, let's see if we can find some underlying structure in the data by dimension reduction, similar to PCA and UMAP, but based on neural networks. 

For this purpose, we build an autoencoder: A model consisting of two neural networks, the encoder and the decoder, that are chained together. The encoder maps the input data to a low-dimension space, from which the decoder maps it back to the original data space. The aim of the model optimization is to reconstrut the original input data with the decoder from the lower-dimensional representation given by the encoder. If this reconstruction is successful, the model will have learned to capture the central structure in the data (based on which it can be accurately reconstructed) in the low-dimension bottleneck layer. 

![](figures/AEwithCellbyGeneMat.png)

Let's build our autoencoder model and train it with our previously defined binary train data `Xtrain`. 

First, we need some functions to initialise our model. We use the hyperbolic tangent as activation function and create an autoencoder with an encoder- and decoder network consisting of two chained layers each:

In [ ]:
struct AE
    encoder
    decoder
end

# function to initialise encoder and decoder 
function AE(in::Int, zdim::Int; 
    hidden::Int=round(Int(in/10)), 
    act_enc::Function=tanh, 
    act_dec::Function=tanh, 
    seed::Int=1234)
    
    Random.seed!(seed)
    encoder = Chain(Dense(in, hidden, act_enc),
                    Dense(hidden, zdim)
    )
    decoder = Chain(Dense(zdim, hidden, act_dec),
                    Dense(hidden, in)
    )
    return AE(encoder, decoder)
end

# define how the model should operate on an input x 
(m::AE)(x) = m.decoder(m.encoder(x))
# automatically collect parameters from the layers included in the AE struct by Flux: 
Flux.Functors.@functor AE

 As our loss function, we use the logit binary cross entropy:

In [ ]:
using Flux: logitbinarycrossentropy
reconstruction_loss(m::AE, x) = logitbinarycrossentropy(m(x), x) 

For simplicity, we dichotomise our data: 

In [ ]:
function dichotomize(x)
	n,p=size(x)
	dicho = zeros(n,p)
	for i=1:p
		dicho[:,i] = ifelse.(x[:,i] .>median(x[:,i]),1.0,0.0)
	end
	return Float32.(dicho)
end

# data
X = dichotomize(mat_norm')' 

# labels
gabagluta_binary = collect(gabagluta.=="GABA")
y = gabagluta_binary

# random permutation of the cells in X
p, n = size(X)
Random.seed!(1234) # for reproducibility 
randindex = Random.randperm(n);

X_perm = X[:, randindex]
y_perm = y[randindex]

# splitting in train and test data
Xtrain = X_perm[:, 1:1000]
Xtest = X_perm[:, 1001:end]

# splitting in train and test labels
ytrain = y_perm[1:1000]
ytest = y_perm[1001:end]

We now define all hyperparameters needed for the training ...

In [ ]:
nepochs = 500 
batchsize = 10
lr = 1e-3

and define and build our model with our previously defined autoencoder-structure by choosing a representative number of latent dimensions `zdim` for our purposes: 

In [ ]:
zdim = 2 
Random.seed!(1234) 
m = AE(ngenes, zdim)

ps = Flux.params(m) # collecting trainable parameters of our model
opt = ADAM(lr) # specifying the optimization algorithm

trainingdata = Flux.DataLoader(Xtrain, batchsize=batchsize, shuffle=true) # generating randomly shuffeled train-batches

After all we can start training the model!

In [ ]:
trainloss = []
testloss = []

for ep in 1:nepochs 
    for batch in trainingdata
        gs = Flux.gradient(ps) do 
            reconstruction_loss(m, batch)
        end
        Flux.Optimise.update!(opt, ps, gs)
    end
    push!(trainloss, reconstruction_loss(m, Xtrain))
    push!(testloss, reconstruction_loss(m, Xtest)) 

    if ep % (nepochs/20) == 0 
        @info "epoch$(ep): current train-loss: $(reconstruction_loss(m, Xtrain)), current test-loss: $(reconstruction_loss(m, Xtest))"
    end

end

After the training we want to get a feeling of how well our model generalizes to unseen data. Therefore we visualize the model loss for train- and test data:

In [ ]:
using Plots
plot(collect(1:nepochs), hcat(log.(10, trainloss .+ 1), log.(10, testloss .+1 ), fill(minimum(log.(10, testloss .+1 )), nepochs)), title = "Loss", label=["train-loss" "test-loss" "min-test-loss"], xlabel="number of epoch", ylabel="log10(model loss)", legend=:topright)

> **Question:**
> 
> What is happening here? What could be the reason for that? 

> **Exercise:** 
>
> How can we prevent our model from overfitting?
 

In [ ]:
@info "Number of epoch with minimum test-loss: $(argmin(log.(10, testloss .+1 )))"

For the next steps, you can re-run the training with a new number of epochs...

Next we want to take a look at the latent space! The following two plots show the latent space for our train data, ...

In [ ]:
zvals_train = m.encoder(Xtrain)
latentrepr = @vlplot(:point, x = zvals_train[1,:], y = zvals_train[2,:], color = {vec(celltype[randindex[1:1000]]), scale={scheme="category20"}})
latentrepr

> **Exercise:**
>
> Plot the `z_vals` color-coded according to GABA- vs. glutamatergic cells! 

> **Exercise:**
>
> Look at the `z_vals` color-coded according to both cell type and GABA- vs. glutamatergic cells also on the test data! 
> 
> Does the model generalise well to unseen data? 

Even if its not as good as with UMAP, the autoencoder is able to learn kind of a structured representation for the data.

## Can we train the autoencoder to look more like UMAP or PCA? 

For that, we need another loss function. We can augment our loss with another term that pushes the learned latent representation more towards the UMAP or PCA representation. You might ask why this would be desirable, if one already has the UMAP solution. Unlike UMAP, the AE gives you a parameteric mapping, so an explicit function to transform general data into the embedding, even if we had new data. 

Before we start, let's again have a look at the previously stored UMAP-representation at beginning of our notebook, for the standardized and log-transformed data matrix: 

In [ ]:
umapplot = @vlplot(:point, x = umap_coords[:,1], y = umap_coords[:,2], color = {vec(celltype), scale={scheme="category20"}})
umapplot

First, we define the new loss function, assuming taking the UMAP/PCA labels as `y`. 

In [ ]:
embedding_loss(m::AE, x, y) = Flux.mse(m.encoder(x), y)
joint_loss(m::AE, x, y) = reconstruction_loss(m, x) + embedding_loss(m, x, y)

Now, we can re-initialize the model parameters, specify our hyperparameters and create the modified training-batches:

In [ ]:
Random.seed!(1234); 
m = AE(ngenes, zdim)
ps = Flux.params(m)

nepochs = 500
lr = 1e-3
batchsize = 100

trainingdata = Flux.DataLoader((Xtrain, umap_coords[randindex[1:1000], :]'), batchsize=batchsize, shuffle=true)

With the generated UMAP data, we are ready to train our modified autoencoder model:

In [ ]:
losslist_joint_train = []
losslist_rec_train = []
losslist_emb_train = []
losslist_joint_test = []
losslist_rec_test = []
losslist_emb_test = []

for ep in 1:nepochs 
    for batch in trainingdata
        gs = Flux.gradient(ps) do 
            joint_loss(m, batch...)
        end
        Flux.Optimise.update!(opt, ps, gs)
    end
    push!(losslist_joint_train, joint_loss(m, Xtrain, umap_coords[randindex[1:1000], :]'))
    push!(losslist_rec_train, reconstruction_loss(m, Xtrain)) 
    push!(losslist_emb_train, embedding_loss(m, Xtrain, umap_coords[randindex[1:1000], :]'))
    push!(losslist_joint_test, joint_loss(m, Xtest, umap_coords[randindex[1001:1525], :]'))
    push!(losslist_rec_test, reconstruction_loss(m, Xtest)) 
    push!(losslist_emb_test, embedding_loss(m, Xtest, umap_coords[randindex[1001:1525], :]'))

    if ep % (nepochs/20) == 0
        @info "epoch$(ep): current train-loss: $(joint_loss(m, Xtrain, umap_coords[randindex[1:1000], :]')), current test-loss: $(joint_loss(m, Xtest, umap_coords[randindex[1001:1525], :]'))"
    end    
end

Let's visualize the different losses from the data we stored during training the autoencoder:

In [ ]:
p1 = plot(collect(1:nepochs), hcat(log.(10, losslist_joint_train .+ 1), log.(10, losslist_joint_test .+1 ), fill(minimum(log.(10, losslist_joint_test .+1 )), nepochs)), title = "joint loss", label=["train-joint loss" "test- joint loss" "min test- joint loss"], xlabel="number of epoch", ylabel="log10(model joint loss)", legend=:topright)
p2 = plot(collect(1:nepochs), hcat(log.(10, losslist_rec_train .+ 1), log.(10, losslist_rec_test .+1 ), fill(minimum(log.(10, losslist_rec_test .+1 )), nepochs)), title = "reconstruction loss", label=["train- rec. loss" "test- rec. loss" "min test- rec. loss"], xlabel="number of epoch", ylabel="log10(model rec. loss)", legend=:topright)
p3 = plot(collect(1:nepochs), hcat(log.(10, losslist_emb_train .+ 1), log.(10, losslist_emb_test .+1 ), fill(minimum(log.(10, losslist_emb_test .+1 )), nepochs)), title = "embedding loss", label=["train- emb. loss" "test- emb. loss" "min test- emb. loss"], xlabel="number of epoch", ylabel="log10(model emb. loss)", legend=:topright)

plot(p1,p2, p3, layout=(1,3), size=(1400,400))

Again, training results in overfitting after a few number of epochs.

> **Exercise:**
> 
> As above, identify the number of epochs for which the different test-losses reach their minimum value. 
>
> Identify a suitable number of epochs and re-train the model for that number of epochs. 
>
> Compare the plots of the latent representation for the test data with a model trained on more vs. less epochs. 
> How do they change? 

In [ ]:
@info "Number of epoch with minimum joint test-loss: $(argmin(log.(10, losslist_joint_test .+1 )))"
@info "Number of epoch with minimum reconstruction test-loss: $(argmin(log.(10, losslist_rec_test .+1 )))"
@info "Number of epoch with minimum embedding test-loss: $(argmin(log.(10, losslist_emb_test .+1 )))"

Finally we have a look at the latent representation and are curious whether this now looks more like the UMAP representation for train and test data:

In [ ]:
zvals_train = m.encoder(Xtrain)
@vlplot(:point, x = zvals_train[1,:], y = zvals_train[2,:], color = vec(gabagluta[randindex[1:1000]]))


In [ ]:
@vlplot(:point, x = zvals_train[1,:], y = zvals_train[2,:], color = {vec(celltype[randindex[1:1000]]), scale={scheme="category20"}})

Quickly compare this to the original UMAP plot and the standard autoencoder-representation from before:

In [ ]:
umapplot

In [ ]:
latentrepr

In [ ]:
zvals_test = m.encoder(Xtest)
@vlplot(:point, x = zvals_test[1,:], y = zvals_test[2,:], color = vec(gabagluta[randindex[1001:1525]]))

In [ ]:
@vlplot(:point, x = zvals_test[1,:], y = zvals_test[2,:], color ={vec(celltype[randindex[1001:1525]]), scale={scheme="category20"}})

> **Note**:
> Choosing a samller number of epochs leads to a better generalization to unseen data. This is one big advantage of the autoencoder model over classical UMAP. 

But if we were just interested in a good visualization of the latent representation on the train set, we can get even closer to the UMAP-representation by choosing a larger number of epochs, for which the joint loss is significantly lower.

If you want to check this out re-run the training with $>=500$ epochs and afterwards re-run the cells with the plots ...

> **Exercise:** 
>
> Do the same with PCA! 

In [ ]:
input_mat = standardize(log.(mat_norm .+ 1)')
pcaout = prcomps(input_mat)
#...


## Finally, let's make it generative!

For simplicity, we first show how to make the VAE work with binary data. 

For the binary version of the VAE, we do again use our dichotomized data `X` from before, meaning that we use `Xtrain` for the training process and later validate the performance of the VAE with `Xtest`.

In [ ]:
@info "Dimensions of Xtrain: $(size(Xtrain))"
@info "Dimensions of Xtest: $(size(Xtest))"
Xtrain

Define VAE struct and initialisation, and some functions for the VAE loss

In [ ]:
struct VAE
    encoder
    μ_encoder
    logσ_encoder
    decoder 
end
Flux.Functors.@functor VAE

# init
function VAE(in::Int, zdim::Int; 
    hidden::Int=round(Int(in/10)), 
    act_enc::Function=tanh, 
    act_dec::Function=tanh, 
    seed::Int=1234)
    
    Random.seed!(seed)
    encoder = Dense(in, hidden, act_enc)
    μ_encoder = Dense(hidden, zdim)
    logσ_encoder = Dense(hidden, zdim)
    decoder = Chain(Dense(zdim, hidden, act_dec),
                    Dense(hidden, in, σ)
    )
    return VAE(encoder, μ_encoder, logσ_encoder, decoder)
end

(m::VAE)(x) = m.decoder(m.μ_encoder(m.encoder(x)))

# loss - components 

sqnorm(x) = sum(abs2, x)
# parameters 
function Flux.params(m::VAE)
    Flux.params(m.encoder, m.μ_encoder, m.logσ_encoder, m.decoder)
end

function latentz(μ::AbstractArray{Float32}, logσ::AbstractArray{Float32})
    return μ .+ exp.(logσ) .* randn(Float32, size(μ))
end

function latentz(μ, logσ)
    return μ + exp(logσ) * randn(Float32)
end

function kl_q_p(μ::AbstractArray{Float32}, logσ::AbstractArray{Float32})
    return 0.5f0 * sum(exp.(2f0 .* logσ) + μ.^2 .- 1f0 .- (2.0f0 .* logσ), dims=1)
end

function logp_x_z(x::AbstractArray{Float32}, dec_z::AbstractArray{Float32})
    return sum(x .* log.(dec_z .+ eps(Float32)) .+ (1.0f0 .- x) .* log.(1.0f0 .- dec_z .+ eps(Float32)), dims=1)
end

# loss - complete functions 
function vae_loss(m::VAE, X::AbstractArray{Float32}) 
    post_μ, post_logσ = m.μ_encoder(m.encoder(X)), m.logσ_encoder(m.encoder(X))
    sample_z = latentz.(post_μ, post_logσ)
    ELBO = logp_x_z(X, m.decoder(sample_z)) .- kl_q_p(post_μ, post_logσ)
    lossval = sum(-ELBO) #+ 0.01f0 * sum(sqnorm, Flux.params(m.decoder))
    return lossval
end

# for callback during training: loss without sampling step 
function determ_vae_loss(m::VAE, X::AbstractArray{Float32}) 
    post_μ, post_logσ = m.μ_encoder(m.encoder(X)), m.logσ_encoder(m.encoder(X))
    ELBO = logp_x_z(X, m.decoder(post_μ)) .- kl_q_p(post_μ, post_logσ) 
    lossval = sum(-ELBO) #+ 0.01f0 * sum(sqnorm, Flux.params(m.decoder))
    return lossval
end

Setting hyperparameters for the training:

In [ ]:
zdim = 2
nepochs=400
lr=1e-3
batchsize=100

Now, let's define the model and train! 


In [ ]:
ngenes, nobs = size(Xtrain)

Random.seed!(1234); 
m = VAE(ngenes, zdim)
ps = Flux.params(m)

opt = ADAM(lr)
trainingdata = Flux.DataLoader(Xtrain, batchsize=batchsize, shuffle=true)

In [ ]:
losslist_train = [] 
losslist_test = []

for ep in 1:nepochs
    for batch in trainingdata
        gs = Flux.gradient(ps) do 
            vae_loss(m, batch)
        end
        Flux.Optimise.update!(opt, ps, gs)
    end
    push!(losslist_train, determ_vae_loss(m, Xtrain))
    push!(losslist_test, determ_vae_loss(m, Xtest))

    if ep % (nepochs/20) == 0 # show loss only at 10 timepoints
        @info "epoch $(ep): current train-loss: $(determ_vae_loss(m, Xtrain)), current test-loss: $(determ_vae_loss(m, Xtest))"
    end    
end

In [ ]:
plot(collect(1:nepochs), hcat(log.(10, losslist_train .+ 1), log.(10, losslist_test .+1 ), fill(minimum(log.(10, losslist_test .+1 )), nepochs)), title = "VAE loss", label=["train-loss" "test-loss" "min test-loss"], xlabel="number of epoch", ylabel="log10(model loss)", legend=:topright)

In [ ]:
# plot latent space 
zvals_train_vae = m.μ_encoder(m.encoder(Xtrain))
@vlplot(:point, x = zvals_train_vae[1,:], y = zvals_train_vae[2,:], color = vec(gabagluta[randindex[1:1000]]))

In [ ]:
@vlplot(:point, x = zvals_train_vae[1,:], y = zvals_train_vae[2,:], color = {vec(celltype[randindex[1:1000]]), scale={scheme="category20"}})

In [ ]:
zvals_test_vae = m.μ_encoder(m.encoder(Xtest))
@vlplot(:point, x = zvals_test_vae[1,:], y = zvals_test_vae[2,:], color = vec(gabagluta[randindex[1001:end]]))

In [ ]:
@vlplot(:point, x = zvals_test_vae[1,:], y = zvals_test_vae[2,:], color = {vec(celltype[randindex[1001:end]]), scale={scheme="category20"}})

## Sample from the trained model!

Now we want to make use of the fact that we have trained a *generative* model, i.e., we want to sample synthetic observations from it. 

In [ ]:
using Distributions
function priorsample(m::VAE, nsamples::Int; seed::Int=6789)
    zdim = size(m.μ_encoder.bias,1)
    z = latentz(zeros(Float32, (zdim, nsamples)), ones(Float32, (zdim, nsamples)))
    decoded_ps = m.decoder(z)
    synthetic_obs = rand.(Bernoulli.(decoded_ps))
    return synthetic_obs
end

function posteriorsample(m::VAE, data; seed::Int=6789)
    post_μ, post_logσ = m.μ_encoder(m.encoder(data)), m.logσ_encoder(m.encoder(data))
    zdim = size(m.μ_encoder.bias,1)
    z = latentz(post_μ, post_logσ)
    decoded_ps = m.decoder(z)
    synthetic_obs = rand.(Bernoulli.(decoded_ps))
    return synthetic_obs
end

In [ ]:
synthetic_data_prior = priorsample(m, 1000)
synthetic_data_post = posteriorsample(m, X)

We can now compare the distribution of some genes in the samples and the original data by looking at the corresponding heatmaps

In [ ]:
using Clustering
using Distances

# original data
distmat_cells = pairwise(cosine_dist, X', dims=1)
hclust_cells = hclust(distmat_cells; linkage=:average, branchorder=:optimal)
distmat_genes = pairwise(cosine_dist, X', dims=2)
hclust_genes = hclust(distmat_genes; linkage=:average, branchorder=:optimal)

# prior sampling
distmat_cells_syn_prior = pairwise(cosine_dist, synthetic_data_prior', dims=1)
hclust_cells_syn_prior = hclust(distmat_cells_syn_prior; linkage=:average, branchorder=:optimal)
distmat_genes_syn_prior = pairwise(cosine_dist, synthetic_data_prior', dims=2)
hclust_genes_syn_prior = hclust(distmat_genes_syn_prior; linkage=:average, branchorder=:optimal)

# posterior sampling
distmat_cells_syn_post = pairwise(cosine_dist, synthetic_data_post', dims=1)
hclust_cells_syn_post = hclust(distmat_cells_syn_post; linkage=:average, branchorder=:optimal)
distmat_genes_syn_post = pairwise(cosine_dist, synthetic_data_post', dims=2)
hclust_genes_syn_post = hclust(distmat_genes_syn_post; linkage=:average, branchorder=:optimal)

plot(heatmap(X'[hclust_cells.order,hclust_genes.order], title = "Original data"),
    heatmap(synthetic_data_prior'[hclust_cells_syn_prior.order, hclust_genes_syn_prior.order], title="Prior sampling"), 
    heatmap(synthetic_data_post'[hclust_cells_syn_post.order, hclust_genes_syn_post.order], title = "Posterior sampling"), 
    layout = (3,1), size=(1000,800)
) 

> **Exercise:** 
> 
> Train the VAE in a supervised way to match the UMAP output!

## Bonus material: How to build a simple classifier from scratch in Julia (even without usig Flux)

This explains how to build a simple neural network from scratch, without relying on the `Flux` library, in Julia. With this we can gain an even deeper understanding of the inner workings of neural networks and their optimisation. 
We will build an example neural network to classify the cells into GABA- and Glutamatergic neurons. 

This is done via *binary classification* by identifying each GABA-eric neuron with a value of 1 and  the Glutamateric neurons with a value of 0.

We start off with a very simple network consisting of just one layer. 

First, we randomly initialise the weight and bias vector: 

In [ ]:
Random.seed!(1234) # for reproducibility 
W = randn(1, ngenes) # output is one-dimensional: GABA vs Gluat
b = randn(1)

We can now define our neural network according to the definition above, but for numerical stability we place the activation function as a part of our loss funtion (*see below*):

In [ ]:
model(x) = W*x .+ b 

For the model output we need values between 0 and 1 to distinguish between the GABA- and the Glutamatergic cells. Therefore we define a function `sigmoid_model` which evaluates the sigmoidal activation of our simple affine `model` function.

We can thus interpret the model output as an estimation by our model of how likely the input is to be a GABA-eric neuron:

In [ ]:
function sigmoid_model(X)
    sigmoid.(model(X))
end

For simplicity, we use the dichotomized version of the gene expression matrix `mat_norm` as input data. This produces a new data matrix `X`, which only contains $0$ and $1$ as values depending on the median value of each row of `mat_norm`. 

Based on this data, the model should be able to learn the GABA- vs. Glutamatergic labels. Additionally, after training our model, we want to figure out how good our modle generalizes to unseen data. Therefore we split the dataset `X` into *train-* and *test datasets* where we use the train data to train our model and the test data to evaluate the performance of the model.

In [ ]:
function dichotomize(x)
	n,p=size(x)
	dicho = zeros(n,p)
	for i=1:p
		dicho[:,i] = ifelse.(x[:,i] .>median(x[:,i]),1.0,0.0)
	end
	return Float32.(dicho)
end

# data
X = dichotomize(mat_norm')' 

# labels
gabagluta_binary = collect(gabagluta.=="GABA")
y = gabagluta_binary

# random permutation of the cells in X
p, n = size(X)
Random.seed!(1234) # for reproducibility 
randindex = Random.randperm(n);

X_perm = X[:, randindex]
y_perm = y[randindex]

# splitting in train and test data
Xtrain = X_perm[:, 1:1000]
Xtest = X_perm[:, 1001:end]

# splitting in train and test labels
ytrain = y_perm[1:1000]
ytest = y_perm[1001:end]

The common loss function for binary classification is the *binary cross entropy*, which measures how well the model output corresponds to the true labels. This also comes built-in with `Flux.jl`. 

As mentioned before we use a slightly modified version of the binary cross entropy, called *logit binary cross entropy*. This version of the binary cross entropy loss is just a composition of the binary cross entropy with the sigmoid activation function $\mathrm{loss} = \mathrm{binarycrossentropy}\circ \sigma$ , where the sigmoid activation (element-wise) is already a part of the loss function. The advantage of this version of the loss function is that it is numerically more stable as for binary cross entropy.

In [ ]:
using Flux: logitbinarycrossentropy
loss(m, X, y) = logitbinarycrossentropy(m(X), y)

Let's see what the model loss currently is on the train data: 

In [ ]:
loss(model, Xtrain, ytrain')

Hm, this doesn't really tell us much. We would additionally want to also monitor the *accuracy*, i.e., how well does the current model classify GABA- vs. Glutamatergic neurons. 

In [ ]:
function crosstab(x, y)
    @assert length(x) == length(y)
    tab = fill(0, (2,2))
    for i in 1:length(x)
        curvals = [x[i] y[i]] .+ 1
        for j in 1:size(tab,1)
            for k in 1:size(tab,2)
                if curvals[1] == j && curvals[2] == k
                    tab[j,k] += 1
                end
            end
        end
    end
    tab
end

function accuracy(m, X, y)
    mx = round.(Int,m(X))
    table = crosstab(mx,y) # tab_1,2 = x=0, y=1; tab_2,1 = x=1,y=0
    sens = table[2,2] / (table[2,2] + table[1,2]) # tp / tp + fn
    spez = table[1,1] / (table[1,1] + table[2,1]) # tn / tn + fp
    acc = (table[1,1] + table[2,2]) / sum(table) # tp + tn / all
    return sens, spez, acc
end

First, let's look at how many cells from the train data were classified into each of the two types:

In [ ]:
model_output = round.(Int,sigmoid_model(Xtrain))
perc_gaba = round(sum(model_output)/length(model_output)*100, digits=2)
@info "$perc_gaba % of all cells classified as GABA-ergic"

# sanity check 
perc_gluta = round(sum(model_output.==0.0)/length(model_output)*100, digits=2)
@info "$perc_gluta % of all cells classified as Glutamatergic"


Now, let's check the accuracy for the train data:

In [ ]:
sens, spez, overall_acc = accuracy(sigmoid_model, Xtrain, ytrain)
sens, spez, overall_acc = accuracy(sigmoid_model, Xtrain, ytrain)

@info "% of cells correctly classified as GABA-ergic:" round(sens*100,digits=2)
@info "% of cells correctly classified as Glutamatergic:" round(spez*100,digits=2)
@info "% of cells classified as Glutamatergic:" round(overall_acc*100,digits=2)

Ok, so at the moment, our network classifies the cells from the training data mostly as Glutamatergic, which is of course not what we want. So we need to train our network, i.e., optimise the loss function with respect to the parameters `W` and `b` of our network. 

Let's see how we can do that using `Flux`. 

To optimise the loss function, we use a stochastic and modified version of gradient descent, specifically implemented in the ADAM optimiser. We thus need to define the learningrate (=stepsize in the gradient descent algorithm), the optimizer and the number of training epochs for which we want to run the stochastic gradient descent algorithm. 
Finally, we need to tell `Flux` which parameters we want to optimise, i.e., with respect to which parameters we would like to get the gradient. 

In [ ]:
using Flux
lr = 0.003 
opt = ADAM(lr)
nepochs = 100  
ps = Flux.params(W, b)

To train the model, we also need to pass the data in the right form. As typically done in neural network training, we partition the data into *batches*. During training, the loss function is calculated for one batch at a time, and after each batch, one parameter update is made by the optimiser. Thus, the parameters are not updated after each individual observation, but after each batch, i.e., each small group of observations. This increases training stability, as single outliers in the data cannot drastically the parameters and the effect of outliers is somewhat mitigated. 

So we need to partition the train data into batches of a pre-specified size, and each batch should contain the data observations, i.e., the gene expression profiles of the cells in the batch, together with the correct GABA- vs. Glutamatergic label. 

In [ ]:
trainingdata = Flux.DataLoader((Xtrain, ytrain'), batchsize=10, shuffle=true)

println("dimensions of the first train-batch for Xtrain:$(size(first(trainingdata)[1]))")
println("dimensions of the first train-batch for ytrain:$(size(first(trainingdata)[2]))")

println("first train-batch:")
first(trainingdata)

Now we can start training! In every epoch, we go through all batches of trainingdata. For each batch, we calculate the gradient of the loss w.r.t the parameters of the model, and update the parameters based on the SDG optimiser. 

During training, we track the loss and the accuracy of the model on the train- and test dataset. 

In [ ]:
# to track loss and accuracy for plotting:
trainlosses = []
trainaccuracies = []
testlosses = []
testaccuracies = []

#train-loop:
for ep in 1:nepochs
    for batch in trainingdata
        gs = Flux.gradient(ps) do 
            loss(model, batch...)
        end
        Flux.Optimise.update!(opt, ps, gs)
    end
    push!(trainlosses, loss(model, Xtrain, ytrain'))
    push!(trainaccuracies, accuracy(sigmoid_model, Xtrain, ytrain')[3])  
    push!(testlosses, loss(model, Xtest, ytest'))
    push!(testaccuracies, accuracy(sigmoid_model, Xtest, ytest')[3]) 
     
    if ep % Int(nepochs/5) == 0 # show loss only at 20 timepoints
        @info "epoch$(ep): current train-loss: $(loss(model, Xtrain, ytrain')), train-accuracy: $(accuracy(sigmoid_model, Xtrain, ytrain')[3]), current test-loss: $(loss(model, Xtest, ytest')), test-accuracy: $(accuracy(sigmoid_model, Xtest, ytest')[3]) ])"
    end    
end

We now visualize the losses and accuracies for train and test data which we collected in the training of our model:

In [ ]:
p1 = plot(collect(1:nepochs), hcat(log.(10, trainlosses .+ 1), log.(10, testlosses .+1 )), title = "Loss", label=["train-loss" "test-loss"], xlabel="number of epoch", ylabel="log10(model loss)", legend=:topright)
p2 = plot(collect(1:nepochs), hcat(trainaccuracies, testaccuracies), title = "Accuracy", label=["train-accuracy" "test-accuracy"], xlabel="number of epoch", ylabel="model accuracy", legend=:bottomright)

plot(p1,p2, layout=(1,2))

This looks nice! The left graphic shows that our modle can reach nearly 0 loss on the train dataset and a very low loss on the test dataset. Also our model can classify nearly any cell from the train- and test data correctly!

You can pick any cell from the test dataset and have a look if our model can classify it correctly:

In [ ]:
#You can pick any number from 1 up to 525 to check our model prediction:
testcell = 112 # test data contains 525 cells

@info "Model prediction for testcell $(testcell): $(1 == Int(round(sigmoid_model(Xtest[:, testcell])[1])) ? "GABAeric" : "Glutamatergic")"
@info "True label of the testcell $(testcell): $(1 == Int(ytest[testcell]) ? "GABAeric" : "Glutamatergic")"

> **Exercise:** How does the number of epochs and the learning rate effect the loss and the accuracy? What happens if you add more layers? 
>
> *Hint 1:* To re-start training from scratch, you have to re-run the cell where you can choose different hyperparameters for the training and re-initialise the parameters for our neural network (*see next code cell*). Choose your preferred hyperparameters and then re-run the training cell (*the one with the train-loop above*).
>
> *Hint 2:* If you want to add more layers, keep in mind that you can place an arbitrary activation function after every middle-layer but not after the output-layer, because we reserve the sigmoid activation function for this one in our binary classifier.

Alternatively, to make writing models easier, we can define a `struct` to represent a fully connected layer and tell Julia how the layer operates on its input: 

In [ ]:
struct FullyConnected
    W
    b 
end

FullyConnected(indim::Int,outdim::Int) = 
    FullyConnected(randn(outdim,indim), randn(outdim))

(l::FullyConnected)(x) = l.W * x .+ l.b

l = FullyConnected(10,5)

This is pretty much exactly what is implemented in the Flux `Dense` layer. Additionally, `Flux` provides convenience functions to quickly construct different layers, chain them together, and automatically extract their parameters for optimisation. 

Written with `Flux.jl`, a simple two-layer model would become:

In [ ]:
model2 = Chain(Dense(ngenes, 100, tanh), Dense(100, 1))